# Experiment B: LLM Context Generation + Selection

In [1]:
import os
import sys


import pandas as pd

from tqdm import tqdm
from itertools import islice

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../utils'))

from llms import TextGenerationModelFactory
from prompting_strategies import ZeroShotPromptFactory, FewShotPromptFactory

/Users/detraviousjamaribrinkley/Documents/Development/classes/cai_6307_nlp/comp_express_mt/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Load Data

In [3]:
file_name = "akuapem_with_tags_dataset-verified_data.xlsx"
path = os.path.join("../data/", file_name)

### Load One to Many

In [4]:
one_to_many_df = pd.read_excel(path, sheet_name="1-M_tags")
akan_source_one = "Akan (Source, One)"
eng_target_many = "English (Target, Many)"
one_many_cols_to_rename = {"Akuapem Twi": akan_source_one, 
                  "English": eng_target_many
                  }
one_to_many_df.rename(columns=one_many_cols_to_rename, inplace=True)
one_to_many_df

,AUD_SIZE,STATUS,AGE,FORMALITY,GENDER,GENDER_2,ANIMACY,SPEECH_ACT,"Akan (Source, One)","English (Target, Many)"
0,INDIVIDUAL,NaN,PEER,INFORMAL,MASCULINE,FEMININE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,He calls her every night.
1,INDIVIDUAL,NaN,PEER,INFORMAL,MASCULINE,MASCULINE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,He calls him every night.
2,INDIVIDUAL,NaN,PEER,INFORMAL,FEMININE,FEMININE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,She calls her every night.
3,INDIVIDUAL,NaN,PEER,INFORMAL,MASCULINE,FEMININE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,He calls her every night.
4,INDIVIDUAL,NaN,PEER,INFORMAL,FEMININE,MASCULINE,ANIMATE,STATEMENT,"Anɔpa yi, ohyiaa no.",She met him this morning.
...,...,...,...,...,...,...,...,...,...,...
458,INDIVIDUAL,NaN,PEER,FORMAL,NEUTRAL,NEUTRAL,INANIMATE,STATEMENT,"Nokwarem no, osu bɛtɔ.",It is definitely going to rain.
459,INDIVIDUAL,NaN,PEER,FORMAL,NEUTRAL,NEUTRAL,INANIMATE,STATEMENT,"Nokwarem no, osu bɛtɔ.",Rain is surely on the way.
460,SMALL GROUP,NaN,PEER,INFORMAL,NEUTRAL,NEUTRAL,ANIMATE,STATEMENT,Ɛsɛ sɛ yehu akokoaa foforo bi a wɔawo no foforo.,We've got to find a new babysitter.
461,SMALL GROUP,NaN,PEER,INFORMAL,NEUTRAL,NEUTRAL,ANIMATE,STATEMENT,Ɛsɛ sɛ yehu akokoaa foforo bi a wɔawo no foforo.,We need to look for another babysitter.


In [5]:
akan_one_to_eng_many_mappings = one_to_many_df.groupby(akan_source_one)[eng_target_many].apply(list).to_dict()
# for akan, e_list in akan_one_to_eng_many_mappings.items():
#     print(f"Key: {akan}")
#     print(f"Values: {e_list}\n")

In [6]:
akan_sentences = list(akan_one_to_eng_many_mappings.keys())
akan_sentences

['Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.',
 'Anadwo biara ɔfrɛ no.',
 'Anigyesɛm ne sɛ, hyɛn no mufo biara anhaw.',
 'Anɔpa yi, ohyiaa no.',
 'Asamaoh ye nipa kese.',
 'Asamoah anhu sɛnea na wobu Esi anim animtiaa no.',
 'Asamoah betumi abua no.',
 'Asamoah yɛ ɔnokwafo.',
 'Awarefo no de sika pii na ɛyɛɛ wɔn fie.',
 'Bere a ade tɔɔ ne so ara pɛ na yɛde no kɔɔ ayaresabea hɔ.',
 'Bere a Ɔhemmaa no soo nkuruwa no mu no, ɔde guu adaka no mu.',
 'Bere bɛn na wiase no bɛba awiei?',
 'Bɔ mmɔden yɛ saa.',
 'Dɛn na wobɛka sɛ ɛsɛ sɛ wokɔ nnwonto no ase?',
 'Efi saa bere no, minni adagyew koraa.',
 'Egye obi na wahu emu biako.',
 'Esi bɔɔ ne tirim sɛ ɔbɛyɛ ne fie ho mfonini.',
 'Esiane ahum no nti, hyɛn no antumi amfi po so hyɛn gyinabea hɔ.',
 "Esiane sɛ na m'ani so atan me nti, anka mereyɛ akyere me.",
 'Hena na ɔka kyerɛɛ wo sɛ mintumi nnom nsu?',
 'Kwaku anyɛ bɔne biara.',
 'Kwaku dii akɔneaba.',
 'Kwaku ntumi ntu ntɛm sɛnea Abena betumi ayɛ no.',
 "M'ani begye ho s

## Initialize Models + Propmt Models

In [7]:
tgmf = TextGenerationModelFactory
llama_31_70b_instruct = tgmf.create_instance('llama-3.1-70b-instruct')
llama_33_70b_instruct = tgmf.create_instance('llama-3.3-70b-instruct')
models = [llama_31_70b_instruct, llama_33_70b_instruct]

In [8]:
def generate_data(prompt: str, model):
    # model_output = model.generate(prompt)
    # print(model_output)
    input_prompt = model.user(prompt)
    raw_text_llm_generation = model.chat_completion([input_prompt])
    # model_outputs[model.model_name] = model_output
    return raw_text_llm_generation

In [28]:
def get_llm_labels(df, prompt_model_col_name):
    # print(prompt_model_col_name)
    filt_llm_name = (df['llm_name'] == prompt_model_col_name)
    # print(filt_llm_name)
    filt_df = df[filt_llm_name]
    # print(filt_df)
    return filt_df['llm_label'], filt_df['llm_sentence'], filt_df['tags']

In [10]:
import json

def parse_context_response(llm_response: str) -> tuple[dict, int]:
    """
    Parse the JSON response from the context prompt.
    
    Parameters
    ----------
    llm_response : str
        The raw LLM response containing JSON.
    
    Returns
    -------
    tuple[dict, int]
        (tags_dict, selection_number)
    
    Example
    -------
    >>> response = '{"tags": {"Audience": "Individual", ...}, "selection": 2}'
    >>> tags, selection = parse_context_response(response)
    >>> print(tags['Audience'])  # "Individual"
    >>> print(selection)  # 2
    """
    # print(llm_response)
    try:
        data = json.loads(llm_response)
        return data['tags'], data['selection']
    except (json.JSONDecodeError, KeyError) as e:
        raise ValueError(f"Failed to parse LLM response: {e}\nResponse: {llm_response}")

In [11]:
zero_shot_context_prompt = ZeroShotPromptFactory("context")
few_shot_context_prompt = FewShotPromptFactory("context")
# chain_of_thought_prompt = ChainOfThoughtPrompt
# prompt_names = [zero_shot_prompt, few_shot_prompt, chain_of_thought_prompt]
# prompt_names = [zero_shot_context_prompt, few_shot_context_prompt]
prompt_names = [few_shot_context_prompt]

In [12]:
mappings = akan_one_to_eng_many_mappings.items()
# mappings
# mappings = islice(akan_one_to_eng_many_mappings.items(), 3)
# type(mappings)

In [13]:
idx = 0
results = {}
for prompt_name in prompt_names:
    prompt_name_and_type = prompt_name.get_name()
    # print(prompt_name_and_type)
    prompt_results = []
    for key, values in tqdm(mappings):
        # print(f"Key: {key}")
        # print(f"\tIndex: {value}")
        
        prompt = prompt_name.get_base_prompt(akan_sentence=key, english_sentences=values)
        if idx == 0:
            print(prompt_name_and_type)
            print(f"Prompt: {prompt}")
            idx = idx + 1

        for model in models:
            llm_result = generate_data(prompt, model)
            # print(f"LLM Result: {llm_result}")
            tags, llm_label = parse_context_response(llm_result)
            # print(f"LLM Label: {llm_label}")
            llm_result_to_sentence = values[llm_label]
            # print(f"Model: {model.__name__()}\tGenerated: {llm_result}\n")
            result = (key, values, int(llm_label), llm_result_to_sentence, tags, f"{prompt_name_and_type}-{model.__name__()}", prompt_name_and_type)
            prompt_results.append(result)
    results[prompt_name] = prompt_results

  0%|          | 0/132 [00:00<?, ?it/s]

few_shot-context
Prompt: You are analyzing an Akan sentence to infer pragmatic context and select the appropriate English translation.
        Example 1:
            Akan: "Ɔyɛ me mpena"
            Options: 1. He is my boyfriend 2. She is my girlfriend 3. They are my lover
            Analysis: The term "mpena" suggests romantic relationship (casual register), "Ɔ" is 3rd person singular

            First, infer the pragmatic context by selecting ONE value for each dimension:
            - Audience: [Individual | Small_Group | Large_Group | Broadcast]
            - Status: [Equal | Superior | Subordinate]
            - Age: [Peer | Elder | Younger]
            - Formality: [Formal | Casual]
            - Gender: [Masculine | Feminine | Neutral]
            - Animacy: [Animate | Inanimate]
            - Speech_Act: [Question | Answer | Statement | Command | Request | Greeting]

            Then, based on these inferred tags, select the most appropriate translation by number.

         

100%|██████████| 132/132 [08:58<00:00,  4.08s/it]


In [14]:
zero_shot_results = results[list(results.keys())[0]]
few_shot_results = results[list(results.keys())[0]]
# chain_of_thought_results = results[list(results.keys())[2]]
col_names = ['akan_sentence', 'english_sentences', 'llm_label', 'llm_sentence', 'tags', 'llm_name', 'prompt_name']

In [15]:
zero_shot_df = pd.DataFrame(zero_shot_results, columns=col_names)
zero_shot_df

,akan_sentence,english_sentences,llm_label,llm_sentence,tags,llm_name,prompt_name
0,Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.,"[The first American colonists arrived in the 17th century., The first settlers came to America in the 1600s., America was first colonized in the 17th century., The first American colonists landed in the 17th century.]",0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
1,Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.,"[The first American colonists arrived in the 17th century., The first settlers came to America in the 1600s., America was first colonized in the 17th century., The first American colonists landed in the 17th century.]",0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
2,Anadwo biara ɔfrɛ no.,"[He calls her every night., He calls him every night., She calls her every night., He calls her every night.]",1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
3,Anadwo biara ɔfrɛ no.,"[He calls her every night., He calls him every night., She calls her every night., He calls her every night.]",1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
4,"Anigyesɛm ne sɛ, hyɛn no mufo biara anhaw.","[ Fortunately, no passengers were injured., Luckily, none of the passengers were hurt., Thankfully, no passengers were harmed., Thankfully, all passengers were safe.]",0,"Fortunately, no passengers were injured.","{'Audience': 'Broadcast', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
...,...,...,...,...,...,...,...
259,Ɛyɛ nwonwa yiye.,"[It's quite remarkable., It is truly amazing., It is quite impressive.]",1,It is truly amazing.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
260,Ɛyɛ pɛ!,"[Perfect!, Spot on!, Exactly!, Just right!]",2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
261,Ɛyɛ pɛ!,"[Perfect!, Spot on!, Exactly!, Just right!]",2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
262,ɔyarehwɛfo no bɛkyerɛ wo ɔkwan a wobɛfa so ayɛ saa.,"[The nurse will tell you how to do it., The nurse will explain the procedure to you., The nurse will show you what to do.]",1,The nurse will explain the procedure to you.,"{'Audience': 'Individual', 'Status': 'Subordinate', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context


In [16]:
few_shot_df = pd.DataFrame(few_shot_results, columns=col_names)
few_shot_df

,akan_sentence,english_sentences,llm_label,llm_sentence,tags,llm_name,prompt_name
0,Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.,"[The first American colonists arrived in the 17th century., The first settlers came to America in the 1600s., America was first colonized in the 17th century., The first American colonists landed in the 17th century.]",0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
1,Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.,"[The first American colonists arrived in the 17th century., The first settlers came to America in the 1600s., America was first colonized in the 17th century., The first American colonists landed in the 17th century.]",0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
2,Anadwo biara ɔfrɛ no.,"[He calls her every night., He calls him every night., She calls her every night., He calls her every night.]",1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
3,Anadwo biara ɔfrɛ no.,"[He calls her every night., He calls him every night., She calls her every night., He calls her every night.]",1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
4,"Anigyesɛm ne sɛ, hyɛn no mufo biara anhaw.","[ Fortunately, no passengers were injured., Luckily, none of the passengers were hurt., Thankfully, no passengers were harmed., Thankfully, all passengers were safe.]",0,"Fortunately, no passengers were injured.","{'Audience': 'Broadcast', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
...,...,...,...,...,...,...,...
259,Ɛyɛ nwonwa yiye.,"[It's quite remarkable., It is truly amazing., It is quite impressive.]",1,It is truly amazing.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
260,Ɛyɛ pɛ!,"[Perfect!, Spot on!, Exactly!, Just right!]",2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context
261,Ɛyɛ pɛ!,"[Perfect!, Spot on!, Exactly!, Just right!]",2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.3-70b-instruct,few_shot-context
262,ɔyarehwɛfo no bɛkyerɛ wo ɔkwan a wobɛfa so ayɛ saa.,"[The nurse will tell you how to do it., The nurse will explain the procedure to you., The nurse will show you what to do.]",1,The nurse will explain the procedure to you.,"{'Audience': 'Individual', 'Status': 'Subordinate', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",few_shot-context-llama-3.1-70b-instruct,few_shot-context


In [17]:
# chain_of_thought_df = pd.DataFrame(chain_of_thought_results, columns=['akan_sentence', 'english_sentences', 'llm_label', 'llm_name', 'prompt_name'])
# chain_of_thought_df

In [18]:
# chain_of_thought_df['llm_label'].values

In [19]:
# chain_of_thought_df['llm_name'].values
# filt_llm_name = (chain_of_thought_df['llm_name'] == f"{prompt_names[2]}_{ model.__name__()}")
# filt_llm_name

In [20]:
prompt_names[0].get_name()

'few_shot-context'

In [29]:
def realign_results(new_df, results_df, prompt_type, models):
    for model in models:
        
        model_name = model.__name__()
        col_prefix = f"{prompt_type}-{model_name}"
        print(f"col_prefix: {col_prefix}")

        llm_labels, llm_sentences, llm_tags = get_llm_labels(results_df, col_prefix)
        # print(f"LLM Labels: {llm_labels}")
        # print(f"LLM Sentences: {llm_sentences}")
        
        new_df[f"{col_prefix}-llm_label"] = llm_labels.values
        new_df[f"{col_prefix}-llm_sentence"] = llm_sentences.values
        new_df[f"{col_prefix}-llm_tags"] = llm_tags.values
        # few_shot_results_df[f"{prompt_names[1]}_{ model.__name__()}"] = few_shot_model_labels.to_numpy().ravel()
        # chain_of_thought_df[model.__name__()] = chain_of_thought_model_labels.to_numpy().ravel()

    return new_df

In [30]:
new_df = pd.DataFrame(akan_sentences, columns=['akan_sentence'])
zero_shot_results_df = realign_results(new_df, zero_shot_df, prompt_names[0].get_name(), models)
zero_shot_results_df

col_prefix: few_shot-context-llama-3.1-70b-instruct
col_prefix: few_shot-context-llama-3.3-70b-instruct


,akan_sentence,few_shot-context-llama-3.1-70b-instruct-llm_label,few_shot-context-llama-3.1-70b-instruct-llm_sentence,few_shot-context-llama-3.1-70b-instruct-llm_tags,few_shot-context-llama-3.3-70b-instruct-llm_label,few_shot-context-llama-3.3-70b-instruct-llm_sentence,few_shot-context-llama-3.3-70b-instruct-llm_tags
0,Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.,0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}"
1,Anadwo biara ɔfrɛ no.,1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
2,"Anigyesɛm ne sɛ, hyɛn no mufo biara anhaw.",0,"Fortunately, no passengers were injured.","{'Audience': 'Broadcast', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",0,"Fortunately, no passengers were injured.","{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
3,"Anɔpa yi, ohyiaa no.",1,She met her this morning.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Feminine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",0,She met him this morning.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Feminine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
4,Asamaoh ye nipa kese.,1,Asamoah is fantastic.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",1,Asamoah is fantastic.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
...,...,...,...,...,...,...,...
127,Ɛsɛ sɛ yɛma yɛn ani da hɔ.,1,We have to stay alert.,"{'Audience': 'Small_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",1,We have to stay alert.,"{'Audience': 'Small_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
128,Ɛyɛ nwonwa sɛ ebetumi aba saa.,1,How could this be possible?,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",1,How could this be possible?,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}"
129,Ɛyɛ nwonwa yiye.,1,It is truly amazing.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",1,It is truly amazing.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}"
130,Ɛyɛ pɛ!,2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Spe

In [31]:
few_shot_results_df = realign_results(new_df, few_shot_df, prompt_names[0].get_name(), models)
few_shot_results_df

col_prefix: few_shot-context-llama-3.1-70b-instruct
col_prefix: few_shot-context-llama-3.3-70b-instruct


,akan_sentence,few_shot-context-llama-3.1-70b-instruct-llm_label,few_shot-context-llama-3.1-70b-instruct-llm_sentence,few_shot-context-llama-3.1-70b-instruct-llm_tags,few_shot-context-llama-3.3-70b-instruct-llm_label,few_shot-context-llama-3.3-70b-instruct-llm_sentence,few_shot-context-llama-3.3-70b-instruct-llm_tags
0,Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.,0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",0,The first American colonists arrived in the 17th century.,"{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}"
1,Anadwo biara ɔfrɛ no.,1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",1,He calls him every night.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
2,"Anigyesɛm ne sɛ, hyɛn no mufo biara anhaw.",0,"Fortunately, no passengers were injured.","{'Audience': 'Broadcast', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",0,"Fortunately, no passengers were injured.","{'Audience': 'Large_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
3,"Anɔpa yi, ohyiaa no.",1,She met her this morning.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Feminine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",0,She met him this morning.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Feminine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
4,Asamaoh ye nipa kese.,1,Asamoah is fantastic.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",1,Asamoah is fantastic.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Masculine', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
...,...,...,...,...,...,...,...
127,Ɛsɛ sɛ yɛma yɛn ani da hɔ.,1,We have to stay alert.,"{'Audience': 'Small_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}",1,We have to stay alert.,"{'Audience': 'Small_Group', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Animate', 'Speech_Act': 'Statement'}"
128,Ɛyɛ nwonwa sɛ ebetumi aba saa.,1,How could this be possible?,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",1,How could this be possible?,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}"
129,Ɛyɛ nwonwa yiye.,1,It is truly amazing.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Formal', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",1,It is truly amazing.,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}"
130,Ɛyɛ pɛ!,2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Speech_Act': 'Statement'}",2,Exactly!,"{'Audience': 'Individual', 'Status': 'Equal', 'Age': 'Peer', 'Formality': 'Casual', 'Gender': 'Neutral', 'Animacy': 'Inanimate', 'Spe

In [24]:
# results_df = pd.concat([zero_shot_results_df, few_shot_results_df])
# results_df